# Gesture game using mediapipe library
This game make use of the mediapipe as the posture detection.


# To install dependencies
- pip install mediapipe  
- pip install numpy  
- pip install cv2  
- pip install pygame

## Declare import and setup game Configuration

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import random 
import time
import pygame
import face_recognition

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pygame.mixer.init()


In [ ]:
# Game Config
RADIUS = 50       # Catch Object Size
DIFFICULTIES = 30 # Min = 0 , Max 100
GAME_TIME = 30    # Game Time in seconds
INCREMENT = 2     # INcrement speed once catch object
MAX_SPEED = 40    # Max speed of the game
INITIAL_SPEED = 25  # Initial speed of the game
IS_DEBUG = False

# Game Constant
HAND = 0
MOUTH = 1

In [ ]:
def initFaceConfig():
    known_faces_config = {
        "Norman" : "resources/norman.png",
        "Karen" : "resources/karen.png",
        "Hailey" : "resources/hailey.jpg"
    }

    name_list = []
    encoding_list = []
    for i in known_faces_config:
        image_file = face_recognition.load_image_file(known_faces_config[i])
        face_encodings = face_recognition.face_encodings(image_file)[0]
        name_list.append(i)
        encoding_list.append(face_encodings)
    
    return { "name" : name_list, "encoding" : encoding_list}

def isInRegion(a,b):
    a = np.array(a) # Point
    b = np.array(b) # Area

    xInRange = b[0] < a[0] < b[1] 
    yInRange = b[2] < a[1] < b[3]

    result = xInRange & yInRange
    return result

def getNewPosition(pos = None):
    while True:
        x = (random.randint(100, 800))
        if (pos[0] - 100 < x < pos[0]+ 100):
            continue
        else:
            break
    return (x, 0)


# Initialize Variable

In [ ]:
is_finish = True
img_x = (random.randint(100, 800))
img_y = 0
score = 0
start_time = 0
speed = 25
seconds = 0
game_type = HAND
known_persons = initFaceConfig()

## Main Logic

In [ ]:
# Start Camera
cap = cv2.VideoCapture(0)   
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally score the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = pose.process(image)
        
        try:
            # Detect the landmarks
            landmarks = results.pose_landmarks.landmark
             
            # Get the Mouth 
            lMouth = [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y * image.shape[0]]
            rMouth = [landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].y * image.shape[0]]
            cMouth = [(lMouth[0]+rMouth[0]) /2, (lMouth[1]+rMouth[1]) /2] 

            # Get Left Hand
            lWrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * image.shape[0]]
            lIndex = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y * image.shape[0]]
            lThumb = [landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y * image.shape[0]]
            lPinky = [landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y * image.shape[0]]

            # Get Right Hand
            rWrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * image.shape[0]]
            rIndex = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y * image.shape[0]]
            rThumb = [landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y * image.shape[0]]
            rPinky = [landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x * image.shape[1],landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y * image.shape[0]]
        
            # Process the game is not finish
            if (not is_finish):

                # Determine whether in Range
                assistance = (100 - DIFFICULTIES)
                area = [img_x, img_x + RADIUS + assistance, img_y, img_y + RADIUS + assistance]  
                
                # Check if Match
                is_win = False
                if (game_type == MOUTH and (is_win or isInRegion(lMouth, area) or isInRegion(rMouth, area) or isInRegion(cMouth, area))): 
                    is_win = True
                    print(1)
                if (game_type == HAND  and (is_win or isInRegion(lWrist, area) or isInRegion(lIndex, area) or isInRegion(lThumb, area) or isInRegion(lPinky, area))):
                    is_win = True
                    print(2)
                if (game_type == HAND  and (is_win or isInRegion(rWrist, area) or isInRegion(rIndex, area) or isInRegion(rThumb, area) or isInRegion(rPinky, area))):
                    is_win = True
                    print(3)
                    
                if (is_win):
                    img_x, img_y = getNewPosition(cMouth)
                    speed += INCREMENT
                    score += 1
                    pygame.mixer.Sound("win.wav").play()
                    
                if (speed > MAX_SPEED):
                    speed = MAX_SPEED

                # Move the Spot
                img_y += speed
                # Handle Exceed Condition
                if (img_y > image.shape[0]):
                    img_y = 10
                    img_x = img_x + 100

                if (img_x > image.shape[1]):
                    img_x = (random.randint(100, 800))

                # Draw the Circle
                center = (int(img_x + RADIUS /2), int(img_y + RADIUS/2))
                cv2.circle(image, center, RADIUS, (245,117,16), -1)    
                    
            # Display Score
            cv2.rectangle(image, (0,0), (100,73), (245,117,16), -1)
            cv2.putText(image, 'Score', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(score), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

            if (not is_finish):
                # Display Time
                now = time.time()
                seconds = GAME_TIME - (now - start_time)

            # Determine Game Finish
            if (seconds <= 0):
                message = "Gameover"
                is_finish = True
                pygame.mixer.music.stop()
            else:
                message = '{:.2f}'.format(seconds) + "s"

            # Print Message 
            cv2.rectangle(image, (image.shape[1] - 200,0), (image.shape[1], 73), (245,117,16), -1)
            cv2.putText(image, message, (image.shape[1] - 180,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            if (is_finish) :
                cv2.rectangle(image, (0, image.shape[0]-100), (200, image.shape[0]), (102,0,255), -1)
                cv2.putText(image, "Hand", (50, image.shape[0]- 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                larea = (0, 200, image.shape[0] - 100, image.shape[0])

                # Right Hand
                if (isInRegion(rWrist, larea) or isInRegion (rIndex, larea) or isInRegion (rThumb, larea) or isInRegion (rPinky, larea)):
                    img_x = (random.randint(100, 800))
                    img_y = 0
                    score = 0
                    start_time = time.time()
                    speed = INITIAL_SPEED
                    is_finish = False
                    pygame.mixer.music.load("background1.mp3")
                    pygame.mixer.music.play()
                    game_type = HAND
                    print("Start Hand Game")
                    continue

                # Left Hand
                cv2.rectangle(image, (image.shape[1] - 200, image.shape[0]-100), (image.shape[1], image.shape[0]), (3,213,123), -1)
                cv2.putText(image, "Mouth", (image.shape[1] - 150, image.shape[0]- 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                rarea = (image.shape[1] - 200, image.shape[1], image.shape[0] - 200, image.shape[0])
        
                if (isInRegion(lWrist, rarea) or isInRegion (lIndex, rarea) or isInRegion (lThumb, rarea) or isInRegion (lPinky, rarea)):
                    img_x = (random.randint(100, 800))
                    img_y = 0
                    score = 0
                    start_time = time.time()
                    speed = INITIA_SPEED
                    is_finish = False
                    pygame.mixer.music.load("background2.mp3")
                    pygame.mixer.music.play()
                    game_type = MOUTH
                    print("Start Mouth Game")
                    continue

        except Exception as e:
            print("Exception occure : ", e)
            pass

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if (IS_DEBUG):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('Eat Together', image)
        
        if cv2.waitKey(5) & 0xFF == 27:
            break
    
cap.release()